## Optimtize To get over 75%!

In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")

from google.colab import drive
drive.mount('/content/drive')

from tensorflow.keras.models import load_model
model = load_model("/content/drive/MyDrive/Colab_Notebooks/deep-learning-challenge/AlphabetSoupCharity.h5")

application_df.head()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [2]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df.drop(['EIN', 'NAME'], axis=1, inplace=True)
application_df.head()

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [3]:
application_types_to_replace = ['T9','T2','T13','T12','T25','T14','T29','T15','T17']

# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
application_df['APPLICATION_TYPE'].value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: APPLICATION_TYPE, dtype: int64

In [4]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
classification_value_counts = application_df['CLASSIFICATION'].value_counts().index.tolist()

classifications_to_replace = classification_value_counts[5:]

# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")

# Check to make sure binning was successful
application_df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: CLASSIFICATION, dtype: int64

In [5]:
application_dummies_df = pd.get_dummies(application_df)

application_dummies_df

,STATUS,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1,5000,1,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1,1,108590,1,0,0,0,1,0,0,0,...,1,0,0,0,0,0,0,0,1,0
2,1,5000,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,1,0
3,1,6692,1,0,0,0,1,0,0,0,...,0,1,0,0,0,0,0,0,1,0
4,1,142590,1,0,0,0,1,0,0,0,...,0,0,1,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34294,1,5000,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,1,0
34295,1,5000,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,1,0
34296,1,5000,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,1,0
34297,1,5000,1,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,1,0


In [6]:
# Split our preprocessed data into our features and target arrays
y = application_dummies_df['IS_SUCCESSFUL']
X = application_dummies_df.drop(columns='IS_SUCCESSFUL')

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## More epochs!

In [ ]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_test_scaled[0])
hidden_nodes_layer1 =  80
hidden_nodes_layer2 = 30


nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, activation="relu", input_dim=number_input_features))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 80)                3520      
                                                                 
 dense_1 (Dense)             (None, 30)                2430      
                                                                 
 dense_2 (Dense)             (None, 1)                 31        
                                                                 
Total params: 5981 (23.36 KB)
Trainable params: 5981 (23.36 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [ ]:
# Train the model with 200 epochs
fit_model = nn.fit(X_train_scaled, y_train, epochs=200)

Epoch 1/200
804/804 [==============================] - 4s 4ms/step - loss: 0.5787 - accuracy: 0.7149
Epoch 2/200
804/804 [==============================] - 3s 4ms/step - loss: 0.5575 - accuracy: 0.7261
Epoch 3/200
804/804 [==============================] - 3s 4ms/step - loss: 0.5537 - accuracy: 0.7294
Epoch 4/200
804/804 [==============================] - 3s 4ms/step - loss: 0.5514 - accuracy: 0.7296
Epoch 5/200
804/804 [==============================] - 3s 4ms/step - loss: 0.5508 - accuracy: 0.7317
Epoch 6/200
804/804 [==============================] - 3s 4ms/step - loss: 0.5487 - accuracy: 0.7326
Epoch 7/200
804/804 [==============================] - 3s 4ms/step - loss: 0.5484 - accuracy: 0.7318
Epoch 8/200
804/804 [==============================] - 3s 4ms/step - loss: 0.5478 - accuracy: 0.7327
Epoch 9/200
804/804 [==============================] - 3s 4ms/step - loss: 0.5466 - accuracy: 0.7331
Epoch 10/200
804/804 [==============================] - 3s 4ms/step - loss: 0.5465 - accura

804/804 [==============================] - 3s 4ms/step - loss: 0.5327 - accuracy: 0.7411
Epoch 161/200
804/804 [==============================] - 3s 4ms/step - loss: 0.5330 - accuracy: 0.7418
Epoch 162/200
804/804 [==============================] - 3s 4ms/step - loss: 0.5330 - accuracy: 0.7411
Epoch 163/200
804/804 [==============================] - 3s 4ms/step - loss: 0.5335 - accuracy: 0.7406
Epoch 164/200
804/804 [==============================] - 2s 3ms/step - loss: 0.5328 - accuracy: 0.7410
Epoch 165/200
804/804 [==============================] - 2s 3ms/step - loss: 0.5328 - accuracy: 0.7411
Epoch 166/200
804/804 [==============================] - 3s 3ms/step - loss: 0.5325 - accuracy: 0.7410
Epoch 167/200
804/804 [==============================] - 2s 3ms/step - loss: 0.5327 - accuracy: 0.7406
Epoch 168/200
804/804 [==============================] - 2s 3ms/step - loss: 0.5339 - accuracy: 0.7412
Epoch 169/200
804/804 [==============================] - 2s 3ms/step - loss: 0.5325 - a

### More layers

In [17]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_test_scaled[0])
hidden_nodes_layer1 =  80
hidden_nodes_layer2 = 30
hidden_nodes_layer3 = 5

nn_2 = tf.keras.models.Sequential()

# First hidden layer
nn_2.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, activation="relu", input_dim=number_input_features))

# Second hidden layer
nn_2.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Third hidden layer
nn_2.add(tf.keras.layers.Dense(units=hidden_nodes_layer3, activation="relu"))

# Output layer
nn_2.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn_2.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_8 (Dense)             (None, 80)                3520      
                                                                 
 dense_9 (Dense)             (None, 30)                2430      
                                                                 
 dense_10 (Dense)            (None, 5)                 155       
                                                                 
 dense_11 (Dense)            (None, 1)                 6         
                                                                 
Total params: 6111 (23.87 KB)
Trainable params: 6111 (23.87 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [18]:
nn_2.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [19]:
fit_model = nn_2.fit(X_train_scaled, y_train, epochs=100)

Epoch 1/100
804/804 [==============================] - 3s 3ms/step - loss: 0.5714 - accuracy: 0.7204
Epoch 2/100
804/804 [==============================] - 3s 3ms/step - loss: 0.5526 - accuracy: 0.7302
Epoch 3/100
804/804 [==============================] - 3s 4ms/step - loss: 0.5504 - accuracy: 0.7311
Epoch 4/100
804/804 [==============================] - 3s 4ms/step - loss: 0.5489 - accuracy: 0.7324
Epoch 5/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5482 - accuracy: 0.7325
Epoch 6/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5470 - accuracy: 0.7345
Epoch 7/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5465 - accuracy: 0.7348
Epoch 8/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5459 - accuracy: 0.7343
Epoch 9/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5453 - accuracy: 0.7333
Epoch 10/100
804/804 [==============================] - 3s 4ms/step - loss: 0.5451 - accura

In [21]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn_2.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 1s - loss: 0.5538 - accuracy: 0.7297 - 773ms/epoch - 3ms/step
Loss: 0.5538005828857422, Accuracy: 0.72967928647995


In [23]:
# Export our model to HDF5 file
nn_2.save('AlphabetSoupCharity_Optimization.h5')

## Bucket the ASK_AMT column


In [7]:
application_df['ASK_AMT'].value_counts()[5:]

7003        2
42072       2
5731        2
46327       2
39122       2
           ..
5371754     1
30060       1
43091152    1
18683       1
36500179    1
Name: ASK_AMT, Length: 8742, dtype: int64

In [8]:
ask_amounts_to_replace = application_df['ASK_AMT'].value_counts().index.tolist()[1:]

# Replace in dataframe
for amt in ask_amounts_to_replace:
    application_df['ASK_AMT'] = application_df['ASK_AMT'].replace(amt,">5000")

# Check to make sure binning was successful
application_df['ASK_AMT'].value_counts()

5000     25398
>5000     8901
Name: ASK_AMT, dtype: int64

In [ ]:
application_dummies_df = pd.get_dummies(application_df)

application_dummies_df

,STATUS,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,APPLICATION_TYPE_T7,...,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y,ASK_AMT_5000,ASK_AMT_>5000
0,1,1,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,1,0
1,1,1,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,1,0,0,1
2,1,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,1,0,1,0
3,1,1,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,1,0,0,1
4,1,1,0,0,0,1,0,0,0,0,...,1,0,0,0,0,0,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34294,1,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,1,0,1,0
34295,1,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,1,0,1,0
34296,1,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,1,0,1,0
34297,1,1,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,1,0,1,0


In [9]:
# Split our preprocessed data into our features and target arrays
y = application_dummies_df['IS_SUCCESSFUL']
X = application_dummies_df.drop(columns='IS_SUCCESSFUL')

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

In [10]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

### More epochs, ASK_AMT bucketed, more nodes

In [ ]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_test_scaled[0])
hidden_nodes_layer1 =  100
hidden_nodes_layer2 = 60


nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, activation="relu", input_dim=number_input_features))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 100)               4500      
                                                                 
 dense_1 (Dense)             (None, 60)                6060      
                                                                 
 dense_2 (Dense)             (None, 1)                 61        
                                                                 
Total params: 10621 (41.49 KB)
Trainable params: 10621 (41.49 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [ ]:
# Train the model
fit_model = nn.fit(X_train_scaled, y_train, epochs=200)

Epoch 1/200
804/804 [==============================] - 1s 898us/step - loss: 0.5682 - accuracy: 0.7216
Epoch 2/200
804/804 [==============================] - 1s 917us/step - loss: 0.5547 - accuracy: 0.7286
Epoch 3/200
804/804 [==============================] - 1s 876us/step - loss: 0.5515 - accuracy: 0.7301
Epoch 4/200
804/804 [==============================] - 1s 935us/step - loss: 0.5502 - accuracy: 0.7307
Epoch 5/200
804/804 [==============================] - 1s 873us/step - loss: 0.5485 - accuracy: 0.7309
Epoch 6/200
804/804 [==============================] - 1s 887us/step - loss: 0.5481 - accuracy: 0.7326
Epoch 7/200
804/804 [==============================] - 1s 892us/step - loss: 0.5467 - accuracy: 0.7321
Epoch 8/200
804/804 [==============================] - 1s 896us/step - loss: 0.5465 - accuracy: 0.7320
Epoch 9/200
804/804 [==============================] - 1s 903us/step - loss: 0.5453 - accuracy: 0.7323
Epoch 10/200
804/804 [==============================] - 1s 903us/step - l

804/804 [==============================] - 1s 865us/step - loss: 0.5325 - accuracy: 0.7396
Epoch 80/200
804/804 [==============================] - 1s 852us/step - loss: 0.5321 - accuracy: 0.7400
Epoch 81/200
804/804 [==============================] - 1s 878us/step - loss: 0.5329 - accuracy: 0.7397
Epoch 82/200
804/804 [==============================] - 1s 878us/step - loss: 0.5327 - accuracy: 0.7393
Epoch 83/200
804/804 [==============================] - 1s 883us/step - loss: 0.5322 - accuracy: 0.7398
Epoch 84/200
804/804 [==============================] - 1s 889us/step - loss: 0.5321 - accuracy: 0.7402
Epoch 85/200
804/804 [==============================] - 1s 871us/step - loss: 0.5325 - accuracy: 0.7409
Epoch 86/200
804/804 [==============================] - 1s 925us/step - loss: 0.5317 - accuracy: 0.7400
Epoch 87/200
804/804 [==============================] - 1s 888us/step - loss: 0.5321 - accuracy: 0.7399
Epoch 88/200
804/804 [==============================] - 1s 874us/step - loss:

804/804 [==============================] - 1s 851us/step - loss: 0.5297 - accuracy: 0.7406
Epoch 158/200
804/804 [==============================] - 1s 857us/step - loss: 0.5299 - accuracy: 0.7410
Epoch 159/200
804/804 [==============================] - 1s 851us/step - loss: 0.5298 - accuracy: 0.7407
Epoch 160/200
804/804 [==============================] - 1s 856us/step - loss: 0.5297 - accuracy: 0.7416
Epoch 161/200
804/804 [==============================] - 1s 853us/step - loss: 0.5293 - accuracy: 0.7418
Epoch 162/200
804/804 [==============================] - 1s 887us/step - loss: 0.5293 - accuracy: 0.7416
Epoch 163/200
804/804 [==============================] - 1s 864us/step - loss: 0.5301 - accuracy: 0.7416
Epoch 164/200
804/804 [==============================] - 1s 857us/step - loss: 0.5295 - accuracy: 0.7419
Epoch 165/200
804/804 [==============================] - 1s 845us/step - loss: 0.5299 - accuracy: 0.7412
Epoch 166/200
804/804 [==============================] - 1s 854us/ste

In [ ]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_test_scaled[0])
hidden_nodes_layer1 =  200
hidden_nodes_layer2 =  120
hidden_nodes_layer3 =  60
hidden_nodes_layer4 =  20

nn_hail_mary = tf.keras.models.Sequential()

# First hidden layer
nn_hail_mary.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, activation="relu", input_dim=number_input_features))

# Second hidden layer
nn_hail_mary.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Third hidden layer
nn_hail_mary.add(tf.keras.layers.Dense(units=hidden_nodes_layer3, activation="relu"))

# Fourth hidden layer
nn_hail_mary.add(tf.keras.layers.Dense(units=hidden_nodes_layer4, activation="relu"))

# Output layer
nn_hail_mary.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn_hail_mary.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_13 (Dense)            (None, 200)               9000      
                                                                 
 dense_14 (Dense)            (None, 120)               24120     
                                                                 
 dense_15 (Dense)            (None, 60)                7260      
                                                                 
 dense_16 (Dense)            (None, 20)                1220      
                                                                 
 dense_17 (Dense)            (None, 1)                 21        
                                                                 
Total params: 41621 (162.58 KB)
Trainable params: 41621 (162.58 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [ ]:
fit_model = nn.fit(X_train_scaled, y_train, epochs=300)

Epoch 1/300
804/804 [==============================] - 1s 1ms/step - loss: 0.5551 - accuracy: 0.7294
Epoch 2/300
804/804 [==============================] - 1s 1ms/step - loss: 0.5444 - accuracy: 0.7365
Epoch 3/300
804/804 [==============================] - 1s 1ms/step - loss: 0.5434 - accuracy: 0.7358
Epoch 4/300
804/804 [==============================] - 1s 1ms/step - loss: 0.5420 - accuracy: 0.7355
Epoch 5/300
804/804 [==============================] - 1s 1ms/step - loss: 0.5417 - accuracy: 0.7352
Epoch 6/300
804/804 [==============================] - 1s 1ms/step - loss: 0.5413 - accuracy: 0.7359
Epoch 7/300
804/804 [==============================] - 1s 1ms/step - loss: 0.5411 - accuracy: 0.7370
Epoch 8/300
804/804 [==============================] - 1s 1ms/step - loss: 0.5409 - accuracy: 0.7378
Epoch 9/300
804/804 [==============================] - 1s 1ms/step - loss: 0.5405 - accuracy: 0.7376
Epoch 10/300
804/804 [==============================] - 1s 1ms/step - loss: 0.5400 - accura

804/804 [==============================] - 1s 1ms/step - loss: 0.5359 - accuracy: 0.7386
Epoch 82/300
804/804 [==============================] - 1s 1ms/step - loss: 0.5365 - accuracy: 0.7379
Epoch 83/300
804/804 [==============================] - 1s 1ms/step - loss: 0.5365 - accuracy: 0.7386
Epoch 84/300
804/804 [==============================] - 1s 1ms/step - loss: 0.5359 - accuracy: 0.7397
Epoch 85/300
804/804 [==============================] - 1s 1ms/step - loss: 0.5361 - accuracy: 0.7395
Epoch 86/300
804/804 [==============================] - 1s 1ms/step - loss: 0.5361 - accuracy: 0.7391
Epoch 87/300
804/804 [==============================] - 1s 1ms/step - loss: 0.5365 - accuracy: 0.7375
Epoch 88/300
804/804 [==============================] - 1s 1ms/step - loss: 0.5361 - accuracy: 0.7373
Epoch 89/300
804/804 [==============================] - 1s 1ms/step - loss: 0.5355 - accuracy: 0.7404
Epoch 90/300
804/804 [==============================] - 1s 1ms/step - loss: 0.5361 - accuracy: 

804/804 [==============================] - 1s 1ms/step - loss: 0.5353 - accuracy: 0.7383
Epoch 161/300
804/804 [==============================] - 1s 1ms/step - loss: 0.5345 - accuracy: 0.7397
Epoch 162/300
804/804 [==============================] - 1s 1ms/step - loss: 0.5346 - accuracy: 0.7393
Epoch 163/300
804/804 [==============================] - 1s 1ms/step - loss: 0.5344 - accuracy: 0.7400
Epoch 164/300
804/804 [==============================] - 1s 1ms/step - loss: 0.5346 - accuracy: 0.7389
Epoch 165/300
804/804 [==============================] - 1s 1ms/step - loss: 0.5351 - accuracy: 0.7382
Epoch 166/300
804/804 [==============================] - 1s 1ms/step - loss: 0.5346 - accuracy: 0.7379
Epoch 167/300
804/804 [==============================] - 1s 1ms/step - loss: 0.5346 - accuracy: 0.7399
Epoch 168/300
804/804 [==============================] - 1s 1ms/step - loss: 0.5344 - accuracy: 0.7395
Epoch 169/300
804/804 [==============================] - 1s 1ms/step - loss: 0.5341 - a

804/804 [==============================] - 1s 1ms/step - loss: 0.5337 - accuracy: 0.7402
Epoch 240/300
804/804 [==============================] - 1s 1ms/step - loss: 0.5341 - accuracy: 0.7381
Epoch 241/300
804/804 [==============================] - 1s 1ms/step - loss: 0.5341 - accuracy: 0.7403
Epoch 242/300
804/804 [==============================] - 1s 1ms/step - loss: 0.5346 - accuracy: 0.7401
Epoch 243/300
804/804 [==============================] - 1s 1ms/step - loss: 0.5339 - accuracy: 0.7405
Epoch 244/300
804/804 [==============================] - 1s 1ms/step - loss: 0.5334 - accuracy: 0.7409
Epoch 245/300
804/804 [==============================] - 1s 1ms/step - loss: 0.5337 - accuracy: 0.7380
Epoch 246/300
804/804 [==============================] - 1s 1ms/step - loss: 0.5352 - accuracy: 0.7391
Epoch 247/300
804/804 [==============================] - 1s 1ms/step - loss: 0.5349 - accuracy: 0.7388
Epoch 248/300
804/804 [==============================] - 1s 1ms/step - loss: 0.5338 - a

In [11]:
# Create a method that creates a new Sequential model with hyperparameter options
def create_model(hp):
    nn_model = tf.keras.models.Sequential()

    number_input_features = len(X_test_scaled[0])

    # Allow kerastuner to decide which activation function to use in hidden layers
    activation = hp.Choice('activation',['relu','tanh','sigmoid'])

    # Allow kerastuner to decide number of neurons in first layer
    nn_model.add(tf.keras.layers.Dense(units=hp.Int('first_units',
        min_value=1,
        max_value=10,
        step=2), activation=activation, input_dim=number_input_features))

    # Allow kerastuner to decide number of hidden layers and neurons in hidden layers
    for i in range(hp.Int('num_layers', 1, 6)):
        nn_model.add(tf.keras.layers.Dense(units=hp.Int('units_' + str(i),
            min_value=1,
            max_value=10,
            step=2),
            activation=activation))

    nn_model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

    # Compile the model
    nn_model.compile(loss="binary_crossentropy", optimizer='adam', metrics=["accuracy"])

    return nn_model

In [13]:
# Import the kerastuner library
!pip install keras_tuner

import keras_tuner as kt

tuner = kt.Hyperband(
    create_model,
    objective="val_accuracy",
    max_epochs=200,
    hyperband_iterations=2)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.9/128.9 kB 3.1 MB/s eta 0:00:00


In [14]:
# Run the kerastuner search for best hyperparameters
tuner.search(X_train_scaled,y_train,epochs=200,validation_data=(X_test_scaled,y_test))

Trial 233 Complete [00h 02m 25s]
val_accuracy: 0.7320116758346558

Best val_accuracy So Far: 0.7341107726097107
Total elapsed time: 02h 45m 06s

Search: Running Trial #234

Value             |Best Value So Far |Hyperparameter
sigmoid           |sigmoid           |activation
7                 |9                 |first_units
6                 |6                 |num_layers
5                 |7                 |units_0
5                 |3                 |units_1
7                 |3                 |units_2
7                 |7                 |units_3
7                 |7                 |units_4
5                 |1                 |units_5
67                |23                |tuner/epochs
23                |0                 |tuner/initial_epoch
2                 |2                 |tuner/bracket
1                 |0                 |tuner/round
0220              |None              |tuner/trial_id

Epoch 24/67
804/804 [==============================] - 5s 5ms/step - loss: 0.5747 - a

KeyboardInterrupt: ignored

In [15]:
# Get best model hyperparameters
best_hyper = tuner.get_best_hyperparameters(1)[0]
best_hyper.values

{'activation': 'sigmoid',
 'first_units': 9,
 'num_layers': 6,
 'units_0': 7,
 'units_1': 3,
 'units_2': 3,
 'units_3': 7,
 'units_4': 7,
 'units_5': 1,
 'tuner/epochs': 23,
 'tuner/initial_epoch': 0,
 'tuner/bracket': 2,
 'tuner/round': 0}

In [16]:
# Evaluate best model against full test data
best_model = tuner.get_best_models(1)[0]
model_loss, model_accuracy = best_model.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 1s - loss: 0.5780 - accuracy: 0.7341 - 656ms/epoch - 2ms/step
Loss: 0.5779968500137329, Accuracy: 0.7341107726097107
